In [37]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
from CKA_functions import adjacency_matrix_motion,adjacency_matrix_distance_motion
import math
import importlib
import RGNN
importlib.reload(RGNN)
from RGNN import ShallowSGCNNet
from torchinfo import summary

In [38]:
in_chans = 32
n_classes = 3
input_window_samples = 400

adj_m,pos = adjacency_matrix_motion()
#print(adj_m)
adj_dis_m, dm = adjacency_matrix_distance_motion(pos,delta=5)
dm
torch_tensor = torch.from_numpy(dm)
edge_weight = torch_tensor.reshape(-1)
print(edge_weight.shape)
model = ShallowSGCNNet(in_chans,n_classes,input_window_samples,edge_weight)

torch.Size([484])


In [39]:
print(model)

ShallowSGCNNet(
  (temporal): Conv2d(1, 10, kernel_size=(1, 25), stride=(1, 1))
  (sgconv): SimpleGCNNet(
    (sgconv): SGConv(37, 2, K=1)
  )
  (batch_norm): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 10), stride=(1, 10), padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=640, out_features=3, bias=True)
)


In [40]:

# Define a threshold distance below which nodes will be connected
threshold = 0  # Adjust as needed

source_nodes = []
target_nodes = []

# Iterate over all elements in the distance matrix, including self-loops and duplicates
for i in range(dm.shape[0]):
    for j in range(dm.shape[1]):  # Iterate over all pairs, including (i, i)
        if dm[i, j] >= threshold:  # If the distance meets the condition
            source_nodes.append(i)  # Source node
            target_nodes.append(j)  # Target node

# Create the edge_index tensor
edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

print("Distance Matrix Shape:", dm.shape)
print("Edge Index:\n", edge_index.shape)

Distance Matrix Shape: (22, 22)
Edge Index:
 torch.Size([2, 484])


In [41]:
print(edge_index.shape)

torch.Size([2, 484])


In [42]:
summary(model,input_size=(2, 32, 400),edge_index=edge_index, col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowSGCNNet                           [2, 32, 400]              [2, 3]                    --                        --
├─Conv2d: 1-1                            [2, 1, 32, 400]           [2, 10, 32, 376]          260                       [1, 25]
├─BatchNorm2d: 1-2                       [2, 10, 32, 376]          [2, 10, 32, 376]          20                        --
├─AvgPool2d: 1-3                         [2, 10, 32, 376]          [2, 10, 32, 37]           --                        [1, 10]
├─SimpleGCNNet: 1-4                      [2, 10, 32, 37]           [2, 10, 32, 2]            484                       --
│    └─SGConv: 2-1                       [2, 10, 32, 37]           [2, 10, 32, 2]            --                        --
│    │    └─SumAggregation: 3-1          [2, 10, 494, 37]          [2, 10, 32, 37]           --                        --
│   